In [143]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
from datetime import time

# CARGA DE LA ACCIDENTALIDAD 
**DESDE EL 2010 al 2018**

**DESDE EL 2019 al 2024**

Fue necesario cargarlo por separado debido al encoding de los csv's.
También fue necesario renombrar las columnas sobre el número de victimas, ya que venían con diferentes nomenclaturas.

In [144]:
# Carga el CSV desde el 2010 al 2018
dfs = []

# Rectifica las columnas nombradas distintas en una sola
nombres_distintos_victimas = {
    'Nº VICTIMAS *': 'Nº VICTIMAS',
    "Nº VICTIMAS *": 'Nº VICTIMAS',
    '* Nº VICTIMAS': 'Nº VICTIMAS',
    "Nº VICTIMAS * ": 'Nº VICTIMAS'
}

for year in range(2010, 2019):
    ubicacion = f"Data/{year}_Accidentalidad.csv"
    df = pd.read_csv(ubicacion, delimiter = ';', encoding = 'ISO-8859-1')
    df.rename(columns = nombres_distintos_victimas, inplace = True)
    
    dfs.append(df)   

df_1 = pd.concat(dfs, ignore_index=True)

# Carga el CSV desde el 2019 al 2024
dfs_2 = []

for year in range(2019, 2025):
    ubicacion = f"Data/{year}_Accidentalidad.csv"
    df = pd.read_csv(ubicacion, delimiter = ';')
    dfs_2.append(df)

df_2 = pd.concat(dfs_2, ignore_index = True)

# ELIMINAR EXPRESIONES REGULARES

In [145]:
# Convertimos columnas y filas a minusculas para una mejor manipulacion
def columnas_minusculas(*dfs):
    for df in dfs:
        df.columns = [columna.lower() for columna in df.columns]
        df[df.select_dtypes(include=['object']).columns] = df.select_dtypes(include=['object']).apply(lambda x: x.str.lower())

columnas_minusculas(df_1, df_2)

In [146]:
# Sustitumos tildes por letras normales
def sustituir_tilde(texto):
    if isinstance(texto, str):
        texto = texto.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    return texto

def quitar_tilde_df(*dfs):
    for df in dfs:    
        df.columns = [sustituir_tilde(columna.lower()) for columna in df.columns]
        for columna in df.select_dtypes(include=['object']).columns:
            df[columna] = df[columna].apply(sustituir_tilde)

quitar_tilde_df(df_1, df_2)

# ELIMINAR y RENOMBRAR COLUMNAS

In [147]:
# Eliminar las columnas 'Unnamed: 19' y 'Unnamed: 20' que se han creado sin ningún dato
df_2.drop(['unnamed: 19', 'unnamed: 20'], axis=1, inplace=True)

# Eliminamos columna cod_lesividad 
df_2.drop(['cod_lesividad'], axis=1, inplace=True)

In [148]:
renombrar_columnas = {
    'estado_meteorologico': 'condicion',
    'lugar accidente'     : 'localizacion',
    'nº'                  : 'numero',
    'tipo vehiculo'       : 'tipo_vehiculo'
}

# Renombrar columnas del primer conjunto (df_1)
df_1.rename(columns = renombrar_columnas, inplace = True)

# Renombrar columnas del segundo conjunto (df_2)
df_2.rename(columns = renombrar_columnas, inplace = True)

In [149]:
df_1.head(2)

,fecha,rango horario,dia semana,distrito,localizacion,numero,nº parte,cpfa granizo,cpfa hielo,cpfa lluvia,...,cpsv grava suelta,cpsv hielo,cpsv seca y limpia,nº victimas,tipo accidente,tipo_vehiculo,tipo persona,sexo,lesividad,tramo edad
0,01/01/2010,de 00:00 a 00:59,viernes,chamartin,calle de cartagena num ...,104,2010/135,no,no,si,...,no,no,no,1,colision doble,turismo,conductor,hombre,il,de 18 a 20 años
1,01/01/2010,de 00:00 a 00:59,viernes,chamartin,calle de cartagena num ...,104,2010/135,no,no,si,...,no,no,no,1,colision doble,turismo,conductor,hombre,il,de 50 a 54 años


In [150]:
df_2.head(2)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,condicion,tipo_vehiculo,tipo_persona,rango_edad,sexo,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2018s017842,04/02/2019,9:10:00,"call. alberto aguilera, 1",1,1.0,centro,colision lateral,despejado,motocicleta > 125cc,conductor,de 45 a 49 años,hombre,asistencia sanitaria solo en el lugar del acci...,"440068,049","4475679,17",n,NaN
1,2018s017842,04/02/2019,9:10:00,"call. alberto aguilera, 1",1,1.0,centro,colision lateral,despejado,turismo,conductor,de 30 a 34 años,mujer,asistencia sanitaria solo en el lugar del acci...,"440068,049","4475679,17",n,NaN


# TRATAR DATOS

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# COLUMNA FECHA

In [151]:
#Convertir fechas a Datetime

def fecha_df_antiguos(*dfs):
    for df in dfs:
        df["fecha"] = pd.to_datetime(df["fecha"], format="%d/%m/%Y")
        df.drop(columns=["dia semana"], inplace=True)

def fecha_df_nuevos(*dfs):
    for df in dfs:
        df["fecha"] = pd.to_datetime(df["fecha"], format="%d/%m/%Y")

fecha_df_antiguos(df_1)
fecha_df_nuevos(df_2)

# COLUMNA RANGO HORARIO
- 2010 - 2018:  Se reemplazan las horas intermedias para obtener un rango de hora más sencillo. 


In [152]:
def primera_hora(rango):
    hora = re.findall(r'\d{1,2}:\d{2}', rango)[0]
    return hora

def aplicar_hora(df):
        df['rango horario'] = df['rango horario'].apply(primera_hora)

aplicar_hora(df_1)

In [153]:
df_1.rename(columns={'rango horario': 'rango_horario'}, inplace=True)

- 2019 - 2024: Se reemplazan las horas intermedias para obtener un rango de hora más sencillo. 

In [154]:
def formatear_horario(horario):
    partes = horario.split(':')
    hora = partes[0]
    minutos = '00'    
    horario_formateado = f"{hora}:{minutos}"
    return horario_formateado

def aplicar_horario(*dfs):
    for df in dfs:
        df["rango_horario"] = df["hora"].apply(formatear_horario)
        del df["hora"]  

aplicar_horario(df_2)

In [155]:
def combinar_fecha_y_rango_horario(df):
    # Agregar segundos a la columna rango horario
    df['rango_horario'] = df['rango_horario'] + ':00'
    # Convertir la columna 'rango horario' a tipo timedelta
    df['rango_horario'] = pd.to_timedelta(df['rango_horario'])

    # Sumar 'fecha' y 'rango horario' para obtener la nueva columna 'fecha_completa'
    df['fecha_completa'] = df['fecha'] + df['rango_horario']
    return df.head(2)

combinar_fecha_y_rango_horario(df_1)
combinar_fecha_y_rango_horario(df_2)

,num_expediente,fecha,localizacion,numero,cod_distrito,distrito,tipo_accidente,condicion,tipo_vehiculo,tipo_persona,rango_edad,sexo,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,rango_horario,fecha_completa
0,2018s017842,2019-02-04,"call. alberto aguilera, 1",1,1.0,centro,colision lateral,despejado,motocicleta > 125cc,conductor,de 45 a 49 años,hombre,asistencia sanitaria solo en el lugar del acci...,"440068,049","4475679,17",n,NaN,0 days 09:00:00,2019-02-04 09:00:00
1,2018s017842,2019-02-04,"call. alberto aguilera, 1",1,1.0,centro,colision lateral,despejado,turismo,conductor,de 30 a 34 años,mujer,asistencia sanitaria solo en el lugar del acci...,"440068,049","4475679,17",n,NaN,0 days 09:00:00,2019-02-04 09:00:00


# COLUMNA DISTRITO
- 2010 - 2018: Eliminar espacios en blanco y nombrar al distrito igual a los otros df


In [156]:
def limpiar_columna_distrito(*dfs):
    for df in dfs:
        df['distrito'] = df['distrito'].str.strip().replace("san blas", "san blas-canillejas")

# Llamada a la función con los DataFrames df_2010, df_2011, ..., df_2018
limpiar_columna_distrito(df_1)

- 2019 - 2024: Eliminar la columna cod_distrito porque no aporta información 

In [157]:
def eliminar_codigo_distrito(*dfs):
    for df in dfs:
        df.drop(columns=["cod_distrito"], inplace=True)
eliminar_codigo_distrito(df_2)

Clasificación de la variable 'distrito' para transformar en dummies:

Centro - Código: 01
Arganzuela - Código: 02
Retiro - Código: 03
Salamanca - Código: 04
Chamartín - Código: 05
Tetuán - Código: 06
Chamberí - Código: 07
Fuencarral-El Pardo - Código: 08
Moncloa-Aravaca - Código: 09

Latina - Código: 10
Carabanchel - Código: 11
Usera - Código: 12
Puente de Vallecas - Código: 13
Moratalaz - Código: 14
Ciudad Lineal - Código: 15
Hortaleza - Código: 16
Villaverde - Código: 17
Villa de Vallecas - Código: 18

Vicálvaro - Código: 19
San Blas-Canillejas - Código: 20
Barajas - Código: 21

# COLUMNA RANGO EDAD
Se cambian los rangos de edades a valores numéricos(int): 
- Menor de 5 años: 1
- De 6 a 9 años: 2
- De 10 a 14 años: 3
- De 15 a 17 años: 4
- De 18 a 20 años: 5
- De 21 a 24 años: 6
- De 25 a 29 años: 7
- De 30 a 34 años: 8
- De 35 a 39 años: 9
- De 40 a 44 años: 10
- De 45 a 49 años: 11
- De 50 a 54 años: 12
- De 55 a 59 años: 13
- De 60 a 64 años: 14
- De 65 a 69 años: 15
- De 70 a 74 años: 16
- Más de 74 años: 17


In [158]:
diccionario_edades = {
    'menor de 5 años'   : 1,
    'de 0 a 5 años'     : 1,
    'de 6 a 9 años'     : 2,
    'de 6 a 9 años '    : 2,
    'de 10 a 14 años'   : 3,
    'de 15 a 17 años'   : 4,
    'de 18 a 20 años'   : 5,
    'de 21 a 24 años'   : 6,
    'de 25 a 29 años'   : 7,
    'de 30 a 34 años'   : 8,
    'de 30 a 34 anos'   : 8,
    'de 35 a 39 años'   : 9,
    'de 40 a 44 años'   : 10,
    'de 45 a 49 años'   : 11,
    'de 50 a 54 años'   : 12,
    'de 55 a 59 años'   : 13,
    'de 60 a 64 años'   : 14,
    'de 65 a 69 años'   : 15,
    'de 70 a 74 años'   : 16,
    'mas de 74 años'    : 17,
    'de mas de 74 años ': 17,
    'de más de 74 años' : 17,
    'desconocido'       : np.nan,
    'desconocida'       : np.nan
}

def reemplazar_edades(df):
        df['rango_edad'] = df['tramo edad'].apply(lambda x : diccionario_edades[x] if x in diccionario_edades and pd.isna(x) == False else x)
        # df['rango_edad'] = pd.to_numeric(df['rango_edad'], errors='coerce').astype('Int64')
        df.drop(columns=["tramo edad"], inplace=True)
        

reemplazar_edades(df_1)

In [159]:

def reemplazar_edades(df):
    df['rango_edad'] = df['rango_edad'].apply(lambda x : diccionario_edades[x] if x in diccionario_edades and pd.isna(x) == False else x)
        # df['rango_edad'] = pd.to_numeric(df['rango_edad'], errors='coerce').astype('Int64')


reemplazar_edades(df_2)

# COLUMNA SEXO
Reemplazar los nulos por nan

In [160]:
def reemplazar_sexo(*dfs):
    for df in dfs:
        df['sexo'] = df['sexo'].replace("desconocido", np.nan)
        df['sexo'] = df['sexo'].replace("no asignado", np.nan)

reemplazar_sexo(df_1, df_2)

# COLUMNA TIPO ACCIDENTE

Se cambian algunas variables dentro de 'tipo_accidente' de ambos Dataframes para mayor homogeneidad en los datos.

In [161]:
tipo_accidente = {
    "colision doble"               : "colision doble",
    "choque con objeto fijo"       : "choque con objeto fijo",
    "atropello"                    : "atropello",
    "otras causas"                 : "otras causas",
    "alcance"                      : "colision doble",
    "colision fronto-lateral"      : "colision doble",
    "choque contra obstaculo fijo" : "choque con objeto fijo",
    "colision lateral"             : "colision doble",
    "otro"                         : "otras causas",
    "atropello a persona"          : "atropello",
    "colision multiple"            : "colision multiple",
    "solo salida de la via"        : "otras causas",
    "colision frontal"             : "colision doble",
    "caida viajero bus"            : "caida",
    "caida ciclomotor"             : "caida",
    "caida motocicleta"            : "caida",
    "caida bicicleta"              : "caida",
    "caida"                        : "caida",
    "vuelco"                       : "vuelco",
    "atropello a animal"           : "atropello",
    "despeñamiento"                : "otras causas"
}

In [162]:
def df_antiguos_accidentes_comunes(*dfs):
    for df in dfs:
        df['tipo_accidente'] = df['tipo accidente'].str.strip().map(tipo_accidente)
        df.drop(columns=["tipo accidente"], inplace=True)
        
df_antiguos_accidentes_comunes(df_1)

In [163]:
def df_nuevos_accidentes_comunes(*dfs):
    for df in dfs:
        df['tipo_accidente'] = df['tipo_accidente'].map(tipo_accidente)
        
df_nuevos_accidentes_comunes(df_2)

In [164]:
df_1["tipo_accidente"].unique()

array(['colision doble', 'colision multiple', 'choque con objeto fijo',
       'caida', 'atropello', 'vuelco', 'otras causas', nan], dtype=object)

In [165]:
df_2["tipo_accidente"].unique()

array(['colision doble', 'choque con objeto fijo', 'caida',
       'otras causas', 'atropello', 'colision multiple', 'vuelco', nan],
      dtype=object)

# COLUMNA TIPO PERSONA

In [166]:
persona_implicada = {
    "conductor": "conductor",
    "pasajero" : "pasajero",
    "peaton"   : "peaton",
    "viajero"  : "pasajero",
    "testigo"  : "peaton",
}

In [167]:
def df_antiguos_implicados(*dfs):
    for df in dfs:
        df['persona_implicada'] = df['tipo persona'].str.strip().map(persona_implicada)
        df.drop(columns=["tipo persona"], inplace=True)
        
df_antiguos_implicados(df_1)

In [168]:
def df_nuevos_implicados(*dfs):
    for df in dfs:
        df['persona_implicada'] = df['tipo_persona'].map(persona_implicada)
        df.drop(columns=["tipo_persona"], inplace=True)
        
df_nuevos_implicados(df_2)

In [169]:
df_1["persona_implicada"].unique()

array(['conductor', 'pasajero', 'peaton'], dtype=object)

In [170]:
df_2["persona_implicada"].unique()

array(['conductor', 'pasajero', 'peaton', nan], dtype=object)

# COLUMNA ESTADO METEOROLÓGICO
Se crean tres rangos, de mejor a peor:
- Favorable
- Desfavorable
- Adverso

In [171]:
columnas_tiempo_borrar = ['cpfa granizo', 
                          'cpfa hielo', 
                          'cpfa lluvia', 
                          'cpfa niebla',
                          'cpfa seco', 
                          'cpfa nieve', 
                          'cpsv mojada', 
                          'cpsv aceite', 
                          'cpsv barro',
                          'cpsv grava suelta', 
                          'cpsv hielo', 
                          'cpsv seca y limpia']

df_tiempo = pd.DataFrame(columns=['Columna', 'Respuesta', 'Conteo'])

# Iterar sobre las columnas de interés y hacer groupby para contar las ocurrencias de cada respuesta
for columna in columnas_tiempo_borrar:
    conteo_respuestas = df_1.groupby(columna).size().reset_index(name='Conteo')
    conteo_respuestas['Columna'] = columna
    conteo_respuestas.rename(columns={columna: 'Respuesta'}, inplace=True)
    df_tiempo = pd.concat([df_tiempo, conteo_respuestas], ignore_index=True)

df_tiempo

,Columna,Respuesta,Conteo
0,cpfa granizo,no,252964
1,cpfa granizo,si,34
2,cpfa hielo,no,252788
3,cpfa hielo,si,210
4,cpfa lluvia,no,224626
5,cpfa lluvia,si,28372
6,cpfa niebla,no,252410
7,cpfa niebla,si,588
8,cpfa seco,no,29255
9,cpfa seco,si,223743


In [172]:
columnas_tiempo = {'cpfa granizo'      : "adverso", 
                   'cpfa hielo'        : "adverso", 
                   'cpfa lluvia'       : "adverso", 
                   'cpfa niebla'       : "desfavorable",
                   'cpfa seco'         : "favorable", 
                   'cpfa nieve'        : "adverso", 
                   'cpsv mojada'       : "desfavorable", 
                   'cpsv aceite'       : "adverso", 
                   'cpsv barro'        : "adverso",
                   'cpsv grava suelta' : "adverso", 
                   'cpsv hielo'        : "adverso", 
                   'cpsv seca y limpia': "favorable"}

# quedarse con la peor: 
prioridades = {"favorable": 1, "desfavorable": 2, "adverso": 3}

def estado_mas_desfavorable(lista):
    return min(lista, key=lambda x: prioridades[x])


def df_antiguos_tiempo(*dfs):
    for df in dfs:
        df['condicion'] = df.apply(lambda x: [columnas_tiempo[col] for col in columnas_tiempo if x[col] == 'si'], axis=1)
        df.drop(columnas_tiempo_borrar, axis=1, inplace=True)
        df['condicion'] = df['condicion'].apply(estado_mas_desfavorable)


df_antiguos_tiempo(df_1)

In [173]:
tiempo= {'despejado'    : "favorable", 
         'nublado'      : "favorable", 
         'lluvia debil' : "desfavorable", 
         'se desconoce' : np.nan,
       'lluvia intensa' : "adverso", 
       'granizando'     : "adverso"}

def df_nuevos_tiempo(*dfs):
    for df in dfs:
        df['condicion'] = df['condicion'].map(tiempo)

df_nuevos_tiempo(df_2)

# VICTIMAS

In [174]:
# Creamos 2 columnas para poder unir 
def df_victimas(df):
        df['victimas'] = np.nan

df_victimas(df_2)

In [175]:
df_1.rename(columns={'nº victimas': 'victimas'}, inplace=True)

# COORDENADAS

In [176]:
# Creamos 2 columnas para poder unir 
def df_antiguos_coordenadas(df):
        df['coordenada_x'] = np.nan
        df['coordenada_y'] = np.nan

df_antiguos_coordenadas(df_1)

In [177]:
df_2.rename(columns={'coordenada_x_utm': 'coordenada_x'}, inplace=True)
df_2.rename(columns={'coordenada_y_utm': 'coordenada_y'}, inplace=True)

# POSITIVO ALCOHOL


In [178]:
# Creamos 2 columnas para poder unir 
def df_antiguos_alcohol_drogas(df):
        df['positiva_alcohol'] = np.nan
        df['positiva_droga'] = np.nan

df_antiguos_alcohol_drogas(df_1)

In [179]:
df_1["positiva_alcohol"].unique()

array([nan])

In [180]:
def df_nuevos_alcohol(*dfs):
    for df in dfs:
        df['positiva_alcohol'] = df['positiva_alcohol'].map({'s': 1, 'n': 0})

df_nuevos_alcohol(df_2)

In [181]:
len(df_2[(df_2['positiva_alcohol'] == 1) & (df_2['positiva_droga'] == 1)])

166

# LESIVIDAD 

In [182]:
df_1["lesividad"].unique()

array(['il                                      ',
       'hl                                      ',
       'hg                                      ', 'no asignada',
       'mt                                      '], dtype=object)

In [183]:
df_2["lesividad"].unique()

array(['asistencia sanitaria solo en el lugar del accidente', nan,
       'ingreso inferior o igual a 24 horas', 'sin asistencia sanitaria',
       'asistencia sanitaria ambulatoria con posterioridad',
       'ingreso superior a 24 horas',
       'atencion en urgencias sin posterior ingreso',
       'asistencia sanitaria inmediata en centro de salud o mutua',
       'fallecido 24 horas', 'se desconoce'], dtype=object)

In [184]:
diccionario_asistencia_sanitaria= {
    'sin asistencia sanitaria'                                 : 'sin asistencia sanitaria',
    'hl'                                                       : 'asistencia sanitaria sin ingreso',
    'asistencia sanitaria inmediata en centro de salud o mutua': 'asistencia sanitaria sin ingreso',
    'asistencia sanitaria ambulatoria con posterioridad'       : 'asistencia sanitaria sin ingreso',
    'asistencia sanitaria solo en el lugar del accidente'      : 'asistencia sanitaria sin ingreso', 
    'atencion en urgencias sin posterior ingreso'              : 'asistencia sanitaria sin ingreso',
    'il'                                                       : 'ingreso inferior 24h',
    'ingreso inferior o igual a 24 horas'                      : 'ingreso inferior 24h',
    'hg'                                                       : 'ingreso superior 24h',
    'ingreso superior a 24 horas'                              : 'ingreso superior 24h',
    'no asignada'                                              : np.nan,
    'se desconoce'                                             : np.nan,
    'mt'                                                       : 'fallecido 24h',
    'fallecido 24 horas'                                       : 'fallecido 24h'
}

In [185]:
def df_antiguos_asistencia(df):
    df['lesividad'] = df['lesividad'].str.strip().map(diccionario_asistencia_sanitaria)

def df_nuevos_asistencia(df):
    df['lesividad'] = df['lesividad'].map(diccionario_asistencia_sanitaria)


In [186]:
df_antiguos_asistencia(df_1)
df_nuevos_asistencia(df_2)

In [187]:
df_1["lesividad"].unique()

array(['ingreso inferior 24h', 'asistencia sanitaria sin ingreso',
       'ingreso superior 24h', nan, 'fallecido 24h'], dtype=object)

In [188]:
df_2["lesividad"].unique()

array(['asistencia sanitaria sin ingreso', nan, 'ingreso inferior 24h',
       'sin asistencia sanitaria', 'ingreso superior 24h',
       'fallecido 24h'], dtype=object)

# ACCIDENTE ID

In [189]:
def generar_accidente_id(df):
    df_1['valor_id'] = df_1['nº parte'].apply(lambda x: x.split('/')[1])
    df_1['año'] = df_1['fecha_completa'].dt.year
    df_1['accidente_id'] = df_1['año'].astype(str) + '.' + df_1['valor_id'].astype(str)
    df_1['accidente_id'] = df_1['accidente_id'].astype(float)
    df_1.drop(columns = ['año', 'valor_id'], axis = 1, inplace = True)

df_1.apply(generar_accidente_id)

fecha                None
rango_horario        None
distrito             None
localizacion         None
numero               None
nº parte             None
victimas             None
tipo_vehiculo        None
sexo                 None
lesividad            None
fecha_completa       None
rango_edad           None
tipo_accidente       None
persona_implicada    None
condicion            None
coordenada_x         None
coordenada_y         None
positiva_alcohol     None
positiva_droga       None
dtype: object

In [190]:
df_1.drop(['nº parte'], axis = 1, inplace = True)

In [191]:
df_1.head(3)

,fecha,rango_horario,distrito,localizacion,numero,victimas,tipo_vehiculo,sexo,lesividad,fecha_completa,rango_edad,tipo_accidente,persona_implicada,condicion,coordenada_x,coordenada_y,positiva_alcohol,positiva_droga,accidente_id
0,2010-01-01,0 days,chamartin,calle de cartagena num ...,104,1,turismo,hombre,ingreso inferior 24h,2010-01-01,5.0,colision doble,conductor,desfavorable,NaN,NaN,NaN,NaN,2010.135
1,2010-01-01,0 days,chamartin,calle de cartagena num ...,104,1,turismo,hombre,ingreso inferior 24h,2010-01-01,12.0,colision doble,conductor,desfavorable,NaN,NaN,NaN,NaN,2010.135
2,2010-01-01,0 days,chamartin,calle de cartagena num ...,104,1,turismo,mujer,asistencia sanitaria sin ingreso,2010-01-01,10.0,colision doble,pasajero,desfavorable,NaN,NaN,NaN,NaN,2010.135


In [192]:
def generar_accidente_ids(df):
    df_2['accidente_id'] = df_2['num_expediente'].apply(lambda x: x.replace('s', '.'))
    df_2['accidente_id'] = df_2['accidente_id'].astype(float)

    df_2.drop('num_expediente', axis = 1, inplace = True)
    
generar_accidente_ids(df_2)

In [193]:
sorted(df_2.columns)

['accidente_id',
 'condicion',
 'coordenada_x',
 'coordenada_y',
 'distrito',
 'fecha',
 'fecha_completa',
 'lesividad',
 'localizacion',
 'numero',
 'persona_implicada',
 'positiva_alcohol',
 'positiva_droga',
 'rango_edad',
 'rango_horario',
 'sexo',
 'tipo_accidente',
 'tipo_vehiculo',
 'victimas']

In [194]:
sorted(df_1.columns)

['accidente_id',
 'condicion',
 'coordenada_x',
 'coordenada_y',
 'distrito',
 'fecha',
 'fecha_completa',
 'lesividad',
 'localizacion',
 'numero',
 'persona_implicada',
 'positiva_alcohol',
 'positiva_droga',
 'rango_edad',
 'rango_horario',
 'sexo',
 'tipo_accidente',
 'tipo_vehiculo',
 'victimas']

In [195]:
df_1.head(3)

,fecha,rango_horario,distrito,localizacion,numero,victimas,tipo_vehiculo,sexo,lesividad,fecha_completa,rango_edad,tipo_accidente,persona_implicada,condicion,coordenada_x,coordenada_y,positiva_alcohol,positiva_droga,accidente_id
0,2010-01-01,0 days,chamartin,calle de cartagena num ...,104,1,turismo,hombre,ingreso inferior 24h,2010-01-01,5.0,colision doble,conductor,desfavorable,NaN,NaN,NaN,NaN,2010.135
1,2010-01-01,0 days,chamartin,calle de cartagena num ...,104,1,turismo,hombre,ingreso inferior 24h,2010-01-01,12.0,colision doble,conductor,desfavorable,NaN,NaN,NaN,NaN,2010.135
2,2010-01-01,0 days,chamartin,calle de cartagena num ...,104,1,turismo,mujer,asistencia sanitaria sin ingreso,2010-01-01,10.0,colision doble,pasajero,desfavorable,NaN,NaN,NaN,NaN,2010.135


In [196]:
df_2.head(3)

,fecha,localizacion,numero,distrito,tipo_accidente,condicion,tipo_vehiculo,rango_edad,sexo,lesividad,coordenada_x,coordenada_y,positiva_alcohol,positiva_droga,rango_horario,fecha_completa,persona_implicada,victimas,accidente_id
0,2019-02-04,"call. alberto aguilera, 1",1,centro,colision doble,favorable,motocicleta > 125cc,11.0,hombre,asistencia sanitaria sin ingreso,"440068,049","4475679,17",0.0,NaN,0 days 09:00:00,2019-02-04 09:00:00,conductor,NaN,2018.017842
1,2019-02-04,"call. alberto aguilera, 1",1,centro,colision doble,favorable,turismo,8.0,mujer,asistencia sanitaria sin ingreso,"440068,049","4475679,17",0.0,NaN,0 days 09:00:00,2019-02-04 09:00:00,conductor,NaN,2018.017842
2,2019-01-01,paseo. santa maria de la cabeza / plaza. eliptica,168,carabanchel,colision doble,NaN,furgoneta,10.0,hombre,NaN,"439139,603","4470836,854",1.0,NaN,0 days 03:00:00,2019-01-01 03:00:00,conductor,NaN,2019.000001


In [197]:
sorted(df_1.columns)

['accidente_id',
 'condicion',
 'coordenada_x',
 'coordenada_y',
 'distrito',
 'fecha',
 'fecha_completa',
 'lesividad',
 'localizacion',
 'numero',
 'persona_implicada',
 'positiva_alcohol',
 'positiva_droga',
 'rango_edad',
 'rango_horario',
 'sexo',
 'tipo_accidente',
 'tipo_vehiculo',
 'victimas']

# Renombrar categorias en tipo_vehiculo


Se crea diccionario para establecer misma categorias entre los Dataframes.

In [198]:
vehiculos= {
    'no asignado'                       : np.nan,
    'sin especificar'                   : np.nan,
    'turismo'                           : "turismo",
    'todo terreno'                      : "turismo",
    'bicicleta'                         : "bicicleta",
    'bicicleta epac (pedaleo asistido)' : 'bicicleta',
    'motocicleta'                       : "motocicleta",
    'motocicleta > 125cc'               : 'motocicleta',
    'motocicleta hasta 125cc'           : 'motocicleta',
    'ciclomotor'                        : "ciclomotor",
    'ciclo'                             : 'ciclomotor',
    'cuadriciclo no ligero'             : 'ciclomotor',
    'cuadriciclo ligero'                : 'ciclomotor',
    'ciclomotor de dos ruedas l1e-b'    : 'ciclomotor',
    'ciclo de motor l1e-a'              : 'ciclomotor',
    'furgoneta'                         : "furgoneta-autocaravana",
    'autocaravana'                      : "furgoneta-autocaravana",
    'caravana'                          : "furgoneta-autocaravana",
    'camion'                            : "camion",
    'camion rigido'                     : 'camion',
    'vehiculo articulado'               : 'camion',
    'tractocamion'                      : 'camion',
    'semiremolque'                      : 'camion',
    'remolque'                          : 'camion',
    'autobus-autocar'                   : "autobus-autocar",
    'autobus'                           : 'autobus-autocar',
    'autobus articulado'                : 'autobus-autocar',
    'microbus <= 17 plazas'             : 'autobus-autocar',
    'ambulancia'                        : "emergencias",
    'ambulancia samur'                  : 'emergencias',
    'camion de bomberos'                : 'emergencias',
    'tren/metro'                        : 'transporte urbano',
    'tranvia'                           : 'transporte urbano',
    'auto-taxi'                         : 'transporte urbano',
    'autobus emt'                       : 'transporte urbano',
    'autobus articulado emt'            : 'transporte urbano',
    'veh.3 ruedas'                      : "vehiculo tres ruedas",
    'moto de tres ruedas > 125cc'       : 'vehiculo tres ruedas',
    'moto de tres ruedas hasta 125cc'   : "vehiculo tres ruedas",
    'ciclomotor de tres ruedas'         : 'vehiculo tres ruedas',
    'varios'                            : "varios",
    'vmu electrico'                     : 'varios',
    'maquinaria de obras'               : 'varios',
    'otros vehiculos sin motor'         : 'varios',
    'otros vehiculos con motor'         : 'varios',
    'patinete no electrico'             : 'varios',
    'patinete'                          : 'varios',
    'maquinaria agricola'               : 'varios',
}

df_1["tipo_vehiculo"] = df_1["tipo_vehiculo"].str.strip().map(vehiculos)
df_2["tipo_vehiculo"] = df_2["tipo_vehiculo"].map(vehiculos)

# ORDEN FINAL COLUMNAS

In [199]:
df_1 = df_1.reindex(columns = ['accidente_id', 'fecha_completa', 'fecha', 'rango_horario', 'localizacion', 'numero', 'distrito', 'coordenada_x', 'coordenada_y', 'condicion', 'lesividad', 'persona_implicada', 'positiva_alcohol','positiva_droga', 'rango_edad', 'sexo', 'tipo_accidente', 'tipo_vehiculo', 'victimas'])

In [200]:
df_2 = df_2[df_1.columns]

In [201]:
df_1.shape

(252998, 19)

In [202]:
df_2.shape

(225785, 19)

In [203]:
df_accidentes = pd.concat([df_1, df_2], ignore_index=True)

In [204]:
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478783 entries, 0 to 478782
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype          
---  ------             --------------   -----          
 0   accidente_id       478783 non-null  float64        
 1   fecha_completa     478783 non-null  datetime64[ns] 
 2   fecha              478783 non-null  datetime64[ns] 
 3   rango_horario      478783 non-null  timedelta64[ns]
 4   localizacion       478783 non-null  object         
 5   numero             478775 non-null  object         
 6   distrito           478775 non-null  object         
 7   coordenada_x       176951 non-null  object         
 8   coordenada_y       176951 non-null  object         
 9   condicion          452103 non-null  object         
 10  lesividad          365297 non-null  object         
 11  persona_implicada  478780 non-null  object         
 12  positiva_alcohol   224988 non-null  float64        
 13  positiva_droga     703 non-nu

In [205]:
df_accidentes.isna().sum()

accidente_id              0
fecha_completa            0
fecha                     0
rango_horario             0
localizacion              0
numero                    8
distrito                  8
coordenada_x         301832
coordenada_y         301832
condicion             26680
lesividad            113486
persona_implicada         3
positiva_alcohol     253795
positiva_droga       478080
rango_edad            44555
sexo                  36182
tipo_accidente           23
tipo_vehiculo         48088
victimas             225785
dtype: int64

In [206]:
(df_accidentes.isna().sum()/df_accidentes.shape[0]*100).sort_values(ascending = False)

positiva_droga       99.853169
coordenada_x         63.041503
coordenada_y         63.041503
positiva_alcohol     53.008357
victimas             47.158107
lesividad            23.703014
tipo_vehiculo        10.043799
rango_edad            9.305886
sexo                  7.557077
condicion             5.572462
tipo_accidente        0.004804
distrito              0.001671
numero                0.001671
persona_implicada     0.000627
fecha_completa        0.000000
localizacion          0.000000
rango_horario         0.000000
fecha                 0.000000
accidente_id          0.000000
dtype: float64

In [207]:
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478783 entries, 0 to 478782
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype          
---  ------             --------------   -----          
 0   accidente_id       478783 non-null  float64        
 1   fecha_completa     478783 non-null  datetime64[ns] 
 2   fecha              478783 non-null  datetime64[ns] 
 3   rango_horario      478783 non-null  timedelta64[ns]
 4   localizacion       478783 non-null  object         
 5   numero             478775 non-null  object         
 6   distrito           478775 non-null  object         
 7   coordenada_x       176951 non-null  object         
 8   coordenada_y       176951 non-null  object         
 9   condicion          452103 non-null  object         
 10  lesividad          365297 non-null  object         
 11  persona_implicada  478780 non-null  object         
 12  positiva_alcohol   224988 non-null  float64        
 13  positiva_droga     703 non-nu

- Convertir rango_horario a datetime

In [208]:
df_accidentes.to_pickle("accidentes.pkl")